## Modelling

We will test different models to see which one produces the best recommendations. All models will be based on KNNs. Since we have no way of telling for sure if a recommendation is good, we will pick 3 poems, read them, and then use them in the model to see if the recommended poems are similar. We will be using Euclidean distances (ED) although we intuitively want to use cosine similarities (CS) because CS does not satisfy the Cauchy-Schwarz inequality (also CS funny enough) and does not make sense to combine with authors and genre. We may do a quick cosine similarity test using only text.

In [1]:
### importing libraries ###

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [2]:
### importing dataframes ###

df = pd.read_csv('data/poem.csv', index_col = 0)
text = pd.read_csv('data/text.csv', index_col = 0)
genre = pd.read_csv('data/genre.csv', index_col = 0)
author = pd.read_csv('data/author.csv', index_col = 0)

Let's make some functions for the models.

In [3]:
### build function ###

# a function to append metrics into pandas dataframe
def features(metrics):
    '''
    function to append metrics
    
    input:
    metrics -> list of features
    
    output: 
    appended pandas dataframe
    '''
    
    feature = metrics[0]
    
    if len(metrics) > 1:
        for metric in metrics[1:]:
            feature = feature.join(metric)
        
    return feature
    

# a function to fit metrics into KNN model
def build_model(metrics):
    '''
    function to build a model
    
    input: 
    metrics -> list of features
    
    output: 
    KNN model
    '''
    
    feature = features(metrics)
    
    model = KNeighborsClassifier().fit(feature, df.index)
    
    return model

# a function to return index of neighbours
def neighbours(metrics, n_neighbours, poem_index):
    '''
    function to return nearest neighbours
    
    input: 
    metrics -> list of features
    n_neighbours -> int number of recommendations
    poem_index -> index of poem
    
    output:
    index of neighbours
    '''
    
    feature = features(metrics)
    
    model = build_model(metrics)
    
    poem = feature.iloc[poem_index:poem_index+1, :]
    
    neighbours = model.kneighbors(X = poem, n_neighbors = n_neighbours + 1)
    
    return neighbours[1][0][1:]

# a function return poem recommendations
def recommend(metrics, n_neighbours, poem_index):
    '''
    function to return recommendations
    
    input: 
    metrics -> list of features
    n_neighbours -> int number of recommendations
    poem_index -> index of poem
    
    output:
    recommendations
    '''
    
    title = df.iloc[poem_index, 0]
    author = df.iloc[poem_index, 1]
    text = df.iloc[poem_index, 3]
    
    print(f'Your poem is \"{title}\" by {author}: \n{text} \n')
    
    indices = neighbours(metrics, n_neighbours, poem_index)
    
    for index in indices:
        title = df.iloc[index, 0]
        author = df.iloc[index, 1]
        text = df.iloc[index, 3]
        
        print(f'A recommended poem is \"{title}\" by {author}: \n{text} \n')

#### Models

Let's test our models.

In [15]:
### genre model ###

# genre
metrics = [genre]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "Portrait of D17: Female, 37 Years Old" by Paisley Rekdal: 
In Phnom Penh’s museum, the skulls are stacked in aquarium tanks: grim toys for hooks to ply free from the rubble. Here, each one gets a tag, a mount, a photograph to suggest a life, perhaps a name, might be envisioned. Yet I’m more moved by what’s anonymous, past; imagine fields of faces sunken with decay, eyes j

In [5]:
### text model ###

# text
metrics = [text]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "Congregations" by Omar Pérez: 
One fisherman alongside the other one seagull alongside the other seagulls over the fishermen.  

A recommended poem is "Bede&#39;s Death Song" by The Venerable Bede: 
Fore there neidfaerae naenig uuiurthit thoncsnotturra than him tharf sie to ymbhycggannae aer his hiniongae huaet his gastae godaes aeththa yflaes aefter deothdaege doemid uue

In [6]:
### author model ###

# author
metrics = [author]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "hill wife" by Jody Gladding: 
beside the grave the ties gave was she there everywhere in the fern or return when he called her of black alder on her lips the fresh chips or felled tree she was free and no child and too wild 

A recommended poem is "Art is an Act" by Jody Gladding: 
not self of violence against will not be gainsaid will brook no argument won't suffer the f

In [7]:
### genre author model ###

# genre
metrics = [genre, author]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "Portrait of D17: Female, 37 Years Old" by Paisley Rekdal: 
In Phnom Penh’s museum, the skulls are stacked in aquarium tanks: grim toys for hooks to ply free from the rubble. Here, each one gets a tag, a mount, a photograph to suggest a life, perhaps a name, might be envisioned. Yet I’m more moved by what’s anonymous, past; imagine fields of faces sunken with decay, eyes j

In [8]:
### genre author model ###

# genre author
metrics = [genre, author * 2]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "hill wife" by Jody Gladding: 
beside the grave the ties gave was she there everywhere in the fern or return when he called her of black alder on her lips the fresh chips or felled tree she was free and no child and too wild 

A recommended poem is "Portrait of D17: Female, 37 Years Old" by Paisley Rekdal: 
In Phnom Penh’s museum, the skulls are stacked in aquarium tanks: 

In [9]:
### genre text model ###

# genre text
metrics = [genre, text]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "Portrait of D17: Female, 37 Years Old" by Paisley Rekdal: 
In Phnom Penh’s museum, the skulls are stacked in aquarium tanks: grim toys for hooks to ply free from the rubble. Here, each one gets a tag, a mount, a photograph to suggest a life, perhaps a name, might be envisioned. Yet I’m more moved by what’s anonymous, past; imagine fields of faces sunken with decay, eyes j

In [10]:
### genre text model ###

# genre text
metrics = [genre * 1.5, text]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "Portrait of D17: Female, 37 Years Old" by Paisley Rekdal: 
In Phnom Penh’s museum, the skulls are stacked in aquarium tanks: grim toys for hooks to ply free from the rubble. Here, each one gets a tag, a mount, a photograph to suggest a life, perhaps a name, might be envisioned. Yet I’m more moved by what’s anonymous, past; imagine fields of faces sunken with decay, eyes j

In [11]:
### text author model ###

# text author
metrics = [text, author]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "Art is an Act" by Jody Gladding: 
not self of violence against will not be gainsaid will brook no argument won't suffer the fools gladly would I climb a mountain of salt with you before dawn lodestar my freight the violent silence the most beautiful word is trespass 

A recommended poem is "hill wife" by Jody Gladding: 
beside the grave the ties gave was she there everywh

In [12]:
### text author model ###

# text author
metrics = [text, author * 2]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "Art is an Act" by Jody Gladding: 
not self of violence against will not be gainsaid will brook no argument won't suffer the fools gladly would I climb a mountain of salt with you before dawn lodestar my freight the violent silence the most beautiful word is trespass 

A recommended poem is "hill wife" by Jody Gladding: 
beside the grave the ties gave was she there everywh

In [13]:
### genre text author model ###

# genre text author
metrics = [genre, text, author]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "Portrait of D17: Female, 37 Years Old" by Paisley Rekdal: 
In Phnom Penh’s museum, the skulls are stacked in aquarium tanks: grim toys for hooks to ply free from the rubble. Here, each one gets a tag, a mount, a photograph to suggest a life, perhaps a name, might be envisioned. Yet I’m more moved by what’s anonymous, past; imagine fields of faces sunken with decay, eyes j

In [14]:
### genre text author model ###

# genre text author
metrics = [genre, text, author * 2]

# recommendations
recommend(metrics, 5, 0)

Your poem is "1-800-FEAR" by Jody Gladding: 
We'd like to talk with you about fear they said so many people live in fear these days they drove up all four of them in a small car nice boy they said beautiful dogs they said so friendly the man ahead of the woman the other two waiting in the drive I was outside digging up the garden no one home I said what are you selling anyway I'm not interested I said well you have a nice day they said here's our card there's a phone number you can call anytime any other houses down this road anyone else live here we'd like to talk to them about living in fear 

A recommended poem is "hill wife" by Jody Gladding: 
beside the grave the ties gave was she there everywhere in the fern or return when he called her of black alder on her lips the fresh chips or felled tree she was free and no child and too wild 

A recommended poem is "Portrait of D17: Female, 37 Years Old" by Paisley Rekdal: 
In Phnom Penh’s museum, the skulls are stacked in aquarium tanks: 

#### Insights

We made the following observations:
* text alone seems to make bad recommendations
* although author is important in making recommendations, since it is only 1 number, we need to increase its weight to x2
* genre seems to be the most useful metric
* combination of all 3 seems to make the best recommendations

We will test the "best" model on two more poems to see if it seems consistent.

In [16]:
### genre text author model ###

# genre text author
metrics = [genre, text, author * 2]

# recommendations
recommend(metrics, 5, 1)

Your poem is "1 January 1965" by Joseph Brodsky: 
The Wise Men will unlearn your name. Above your head no star will flame. One weary sound will be the same— the hoarse roar of the gale. The shadows fall from your tired eyes as your lone bedside candle dies, for here the calendar breeds nights till stores of candles fail. What prompts this melancholy key? A long familiar melody. It sounds again. So let it be. Let it sound from this night. Let it sound in my hour of death— as gratefulness of eyes and lips for that which sometimes makes us lift our gaze to the far sky. You glare in silence at the wall. Your stocking gapes: no gifts at all. It's clear that you are now too old to trust in good Saint Nick; that it's too late for miracles. —But suddenly, lifting your eyes to heaven's light, you realize: your life is a sheer gift. 

A recommended poem is "Burning the Old Year" by Naomi Shihab Nye: 
Letters swallow themselves in seconds. Notes friends tied to the doorknob, transparent scarlet p

In [17]:
### genre text author model ###

# genre text author
metrics = [genre, text, author * 2]

# recommendations
recommend(metrics, 5, 2)

Your poem is "10-Year-Old Shot Three Times, but She’s Fine" by Patricia Smith: 
Dumbfounded in hospital whites, you are picture-book itty-bit, floundering in bleach and steel. Braids untwirl and corkscrew, you squirm, the crater in your shoulder spews a soft voltage. On a TV screwed into the wall above your head, neon rollicks. A wide-eyed train engine perfectly smokes, warbles a song about forward. Who shot you, baby? I don’t know. I was playing. You didn’t see anyone? I was playing with my friend Sharon. I was on the swing and she was— Are you sure you didn’t— No, I ain’t seen nobody but Sharon. I heard people yelling though, and— Each bullet repainted you against the brick, kicked you a little sideways, made you need air differently. You leaked something that still goldens the boulevard. I ain’t seen nobody, I told you. And at A. Lincoln Elementary on Washington Street, or Jefferson Elementary on Madison Street, or Adams Elementary just off the Eisenhower Expressway, we gather the i

#### Conclusions:
The final model using text, genre, and author x 2 seems to be the best of all the models and gives good poetry recommendations.